# Projeto Estatística Descritiva de Dados
## Lucas Natan Correia Couri e Mariama Oliveira

# Análise exploratória

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import researchpy as rp
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from IPython.core.interactiveshell import InteractiveShell
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
dados = pd.read_csv("dataset_stroke.csv")

In [ ]:
dados.head()

In [ ]:
dados.shape

## Pré-processamento de dados

In [ ]:
dados.dtypes

Convertendos tipos "object" para "category"

In [ ]:
dados['gender'] = dados['gender'].astype('category')
dados['ever_married'] = dados['ever_married'].astype('category')
dados['work_type'] = dados['work_type'].astype('category')
dados['Residence_type'] = dados['Residence_type'].astype('category')
dados['smoking_status'] = dados['smoking_status'].astype('category')
dados['hypertension'] = dados['hypertension'].astype('category')
dados['heart_disease'] = dados['heart_disease'].astype('category')
dados['stroke'] = dados['stroke'].astype('category')


In [ ]:
dados[(dados['age']<2) & (dados['stroke']==0)]

In [ ]:
dados.describe()

In [ ]:
dados['bmi'].plot.box()

### **Verificando dados nulos**

In [ ]:
dados.isnull().sum()/len(dados)

In [ ]:
dados['gender'].value_counts() 
dados['ever_married'].value_counts()
dados['work_type'].value_counts()
dados['Residence_type'].value_counts()
dados['smoking_status'].value_counts()

In [ ]:
dados[['hypertension', 'heart_disease','stroke']].apply(pd.Series.value_counts)

In [ ]:
dados[(dados['bmi'].isnull()) & (dados['stroke']==1)].shape

In [ ]:
dados[(dados['stroke']==1)].shape

### Iremos retirar as instâncias que apresentam valores nulos para bmi (IMC). Futuramente, iremos comparar se caso esse atributo for inputado, o classificador apresentará uma melhor performance.

In [ ]:
dados.dropna(inplace = True)
#dados['bmi'] = dados['bmi'].fillna(dados['bmi'].mean())

In [ ]:
dados[dados.duplicated()]

In [ ]:
dados[ ( dados['smoking_status']=="Unknown" ) & ( dados['stroke']==1 ) ].shape

In [ ]:
#dados.shape #4909
dados = dados[dados['smoking_status']!="Unknown"]

In [ ]:
dados = dados[dados['gender']!="Other"]

## Discretizando variáveis contínuas

In [ ]:
dados['faixaIdade'] = pd.cut(dados['age'], [0, 10, 25, 50, 75, 90])
dados['faixaIMC'] = pd.cut(dados['bmi'], [0, 18.5, 25, 30, 40, 100])
dados['faixaGlicose'] = pd.cut(dados['avg_glucose_level'], [0, 70, 100, 125, 300])
#https://www.who.int/data/gho/indicator-metadata-registry/imr-details/2380
dados['faixaIdade'] = dados['faixaIdade'].astype('category')
dados['faixaIMC'] = dados['faixaIMC'].astype('category')
dados['faixaGlicose'] = dados['faixaGlicose'].astype('category')


# Análise

Verificando distribuição dos atributos

In [ ]:
# ax = sns.countplot(x="stroke", data=dados)
#ax = sns.barplot(x="day", y="total_bill", hue="sex", data=tips)


In [ ]:
#dados['hypertension'] 
#dados['heart_disease']
#dados['stroke']
dados[['hypertension', 'heart_disease','stroke']].apply(pd.Series.value_counts)

In [ ]:
sns.countplot(x="heart_disease", data=dados[dados['stroke']==1])

In [ ]:
sns.histplot(data=dados, x="age")

In [ ]:
sns.histplot(data=dados, x="avg_glucose_level")

In [ ]:
sns.histplot(data=dados, x="bmi")

In [ ]:
sns.scatterplot(data=dados, x="age", y="bmi", hue=dados['stroke'])

In [ ]:
sns.scatterplot(data=dados, x="bmi", y="avg_glucose_level", hue=dados['stroke'])

In [ ]:
### Fazer os graficos para as categoricas (4 plots por janela)
### Fazer os graficos para as categoricas apenas para stroke==1

In [ ]:
dados.head()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharey=False)

sns.countplot(ax=axes[0,0], x="heart_disease", data=dados)
axes[0,0].tick_params(rotation=90, axis='x')
#axes[0,0].set_title(nomes_series[0])
#axes[0,0].set_ylabel('Número Total de Twitts')
#axes[0,0].set_xlabel('Dia')

sns.countplot(ax=axes[0,1], x="hypertension", data=dados)
axes[0,1].tick_params(rotation=90, axis='x')
#axes[0,1].set_title(nomes_series[1])
#axes[0,1].set_ylabel('Número Total de Twitts')
#axes[0,1].set_xlabel('Dia')

sns.countplot(ax=axes[1,0], x="faixaIdade", data=dados)
axes[1,0].tick_params(rotation=90, axis='x')
#axes[1,0].set_title(nomes_series[2])
#axes[1,0].set_ylabel('Número Total de Twitts')
#axes[1,0].set_xlabel('Dia')

sns.countplot(ax=axes[1,1], x="faixaGlicose", data=dados)
axes[1,1].tick_params(rotation=90, axis='x')
#axes[1,1].set_title(nomes_series[3])
#axes[1,1].set_ylabel('Número Total de Twitts')
#axes[1,1].set_xlabel('Dia')

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharey=False)

sns.countplot(ax=axes[0,0], x="heart_disease", data=dados[dados['stroke']==1])
axes[0,0].tick_params(rotation=90, axis='x')
#axes[0,0].set_title(nomes_series[0])
#axes[0,0].set_ylabel('Número Total de Twitts')
#axes[0,0].set_xlabel('Dia')

sns.countplot(ax=axes[0,1], x="hypertension", data=dados[dados['stroke']==1])
axes[0,1].tick_params(rotation=90, axis='x')
#axes[0,1].set_title(nomes_series[1])
#axes[0,1].set_ylabel('Número Total de Twitts')
#axes[0,1].set_xlabel('Dia')

sns.countplot(ax=axes[1,0], x="faixaIdade", data=dados[dados['stroke']==1])
axes[1,0].tick_params(rotation=90, axis='x')
#axes[1,0].set_title(nomes_series[2])
#axes[1,0].set_ylabel('Número Total de Twitts')
#axes[1,0].set_xlabel('Dia')

sns.countplot(ax=axes[1,1], x="faixaGlicose", data=dados[dados['stroke']==1])
axes[1,1].tick_params(rotation=90, axis='x')
#axes[1,1].set_title(nomes_series[3])
#axes[1,1].set_ylabel('Número Total de Twitts')
#axes[1,1].set_xlabel('Dia')

In [ ]:
dados.corr(method='pearson')

In [ ]:
correlation_mat = dados.drop(columns = "id").corr()

sns.heatmap(correlation_mat, annot = True)

plt.show()

In [ ]:
varCategoricas = ['gender', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'smoking_status', 'Residence_type', 'faixaIdade', 'faixaIMC', 'faixaGlicose']

pvalores = []

for variavel in varCategoricas:
    crosstab = pd.crosstab(dados["stroke"], dados[variavel])
    pvalor = stats.chi2_contingency(crosstab)[1]
    pvalores.append(pvalor)
    print(variavel, pvalor)

In [ ]:
crosstab, test_results, expected = rp.crosstab(dados["stroke"], dados["heart_disease"],
                                               test= "chi-square",
                                               expected_freqs = True,
                                               prop= "cell")

#crosstab
test_results

## Discretizando variáveis contínuas

In [ ]:
dados['faixaIdade'] = pd.cut(dados['age'], [0, 10, 25, 50, 75, 90])

In [ ]:
dados['faixaIMC'] = pd.cut(dados['bmi'], [0, 18.5, 25, 30, 40, 100])

In [ ]:
dados['faixaGlicose'] = pd.cut(dados['avg_glucose_level'], [0, 70, 100, 125, 300])
#https://www.who.int/data/gho/indicator-metadata-registry/imr-details/2380

In [ ]:
dados['faixaIdade'] = dados['faixaIdade'].astype('category')
dados['faixaIMC'] = dados['faixaIMC'].astype('category')
dados['faixaGlicose'] = dados['faixaGlicose'].astype('category')

# Classificador Naive Bayes

## Com scikit

Separando em dados de treinamento e teste

In [ ]:
dados.head()

In [ ]:
X = dados.drop(columns='stroke')

y = dados['stroke']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



Codificando as variáveis

In [ ]:
X_trainNB = X_train
X_testNB = X_test
#X_trainNB
#X_testNB

In [ ]:
cat_columns = X_trainNB.select_dtypes(['category']).columns
X_trainNB[cat_columns] = X_trainNB[cat_columns].apply(lambda x: x.cat.codes)

cat_columns = X_testNB.select_dtypes(['category']).columns
X_testNB[cat_columns] = X_testNB[cat_columns].apply(lambda x: x.cat.codes)

X_trainNB = X_trainNB.drop(columns=["age", "bmi", "avg_glucose_level", "id"])
X_testNB = X_testNB.drop(columns=["age", "bmi", "avg_glucose_level", "id"])




In [ ]:
# With sklearn
ctgrclNB = CategoricalNB(alpha=1)
ctgrclNB.fit(X_trainNB, y_train)
ctgrclNB.predict_proba(np.array(X_testNB))

In [ ]:
y_pred = ctgrclNB.predict(np.array(X_testNB))

In [ ]:
f1_score(y_test, y_pred, average=None)
# accuracy_score(y_test, y_pred)

#res_df = pd.DataFrame(resultado_predicao, y_test)

In [ ]:
confusion_matrix(y_test, y_pred) #tn, fp, fn, tp

In [ ]:
target_names = ['no stroke', 'stroke']
print(classification_report(y_test, y_pred, target_names=target_names))

## Sem scikit

In [ ]:
trainNB = X_trainNB
trainNB['stroke'] = y_train
trainNB.head()

In [ ]:
def calculoProbCond(entradaTestar, conjuntoTreino):
    
    arrayProbabilidades = [[],[]]
    for nomeColuna in conjuntoTreino.columns: #exceto stroke?

        if nomeColuna!="stroke":   
            valorTestar = entradaTestar[nomeColuna] #Ver se precisa passar o valor da linha (0)
 
            atributoNao = len(conjuntoTreino[(conjuntoTreino[nomeColuna]==valorTestar)&(conjuntoTreino['stroke']==0)]) / len(conjuntoTreino[conjuntoTreino['stroke']==0])
            atributoSim = len(conjuntoTreino[(conjuntoTreino[nomeColuna]==valorTestar)&(conjuntoTreino['stroke']==1)]) / len(conjuntoTreino[conjuntoTreino['stroke']==1])

            arrayProbabilidades[0].append(atributoNao)
            arrayProbabilidades[1].append(atributoSim)

        #genderp0 = (len(XcomY[(XcomY['gender']==1)&(XcomY['stroke']==0)])+1) / (len(XcomY[XcomY['Y']==0])+(1*5))
        #ctgrclX15p1 = (len(XcomY[(XcomY['gender']==1)&(XcomY['stroke']==1)])+1) / (len(XcomY[XcomY['Y']==1])+(1*5))


    return arrayProbabilidades




In [ ]:
def calculoProbabilidadePred(listaProbabilidades, conjuntoTreino):

    Likelihood0 = np.prod(listaProbabilidades[0])
    Likelihood1 = np.prod(listaProbabilidades[1])

    ProbStroke0 = len(conjuntoTreino[conjuntoTreino['stroke']==0])/len(conjuntoTreino)
    ProbStroke1 = len(conjuntoTreino[conjuntoTreino['stroke']==1])/len(conjuntoTreino)


    ProbTotal = (ProbStroke0*Likelihood0) + (ProbStroke1*Likelihood1)

    probNao = (ProbStroke0*Likelihood0)/ProbTotal
    probSim = (ProbStroke1*Likelihood1)/ProbTotal
    return 0 if probNao>probSim else 1

In [ ]:
##Iterar sobre a base de teste
def predictStroke(conjuntoTreino, conjuntoTeste) :

    arrayPred = []
    ##Loop que determina todas as predicoes
    for index, linha in conjuntoTeste.iterrows():
        resposta = calculoProbPosteriori(linha, conjuntoTreino)
        predicao = calculoProbabilidadePred(resposta, conjuntoTreino)
        print(index , ": ", predicao)
        arrayPred.append(predicao)
   
    return arrayPred

In [ ]:
pred_Y  =  predictStroke(trainNB, X_test)


In [ ]:
pred_Y

In [ ]:
accuracy_score(y_test, pred_Y)

In [ ]:
f1_score(y_test, pred_Y, average=None)
#([0.9604299 , 0.12903226]) resultado do scikit

In [ ]:
confusion_matrix(y_test, pred_Y) #tn, fp, fn, tp f1=tp/tp+1/2(fp+fn) f0=942/942+1/2(27+53) e f1=6/6+1/2(27+53)

In [ ]:
target_names = ['no stroke', 'stroke']
print(classification_report(y_test, pred_Y, target_names=target_names))

## KNN

In [ ]:
X_trainKN = X_train.drop(columns = ["faixaIdade", "faixaIMC", "faixaGlicose", "id"])
X_testKN = X_test.drop(columns = ["faixaIdade", "faixaIMC", "faixaGlicose", "id"])

In [ ]:
train_dummies = pd.get_dummies(X_trainKN[["work_type", "smoking_status"]].astype(str))
test_dummies = pd.get_dummies(X_testKN[["work_type", "smoking_status"]].astype(str))

In [ ]:
X_trainKN = X_trainKN.join(train_dummies)
X_testKN = X_testKN.join(test_dummies)

In [ ]:
knnClassifier = KNeighborsClassifier(n_neighbors=1)
knnClassifier.fit(X_trainKN, y_train)
pred_KN = knnClassifier.predict(X_testKN)


In [ ]:
f1_score(y_test, pred_KN, average=None)

In [ ]:
accuracy_score(y_test, pred_KN)

In [ ]:
confusion_matrix(y_test, pred_KN) #tn, fp, fn, tp

In [ ]:
target_names = ['no stroke', 'stroke']
print(classification_report(y_test, pred_KN, target_names=target_names))

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, pred_KN).ravel()
tn, fp, fn, tp

In [ ]:
len(dados[dados['stroke']==1])
len(dados)#[dados['stroke']==1])

# Experimento balanceamento

In [ ]:
avcSim = dados[dados['stroke']==1]
avcNao = dados[dados['stroke']==0]
avcNao = avcNao.sample(frac=0.0555, replace=False, random_state=195)

In [ ]:
len(avcNao)
len(avcSim)

In [ ]:
balanceado = avcSim.append(avcNao)

In [ ]:
Xb = balanceado.drop(columns='stroke')

yb = balanceado['stroke']

Xb_train, Xb_test, yb_train, yb_test = train_test_split(X, y, test_size=0.3, random_state=42)


## Naive Bayes

## Knn